In [ ]:
# --- SECTION 1: Setup and Imports ---
# This code is designed to be run in Google Colab.

# Install necessary libraries (run this block first in Colab)
# !pip install -q gradio pandas requests

import gradio as gr
import pandas as pd
import requests
import json
import time
import os

# --- SECTION 2: Configuration and Data Loading ---

# CRITICAL: Replace "YOUR_GEMINI_API_KEY" with your actual key
# You can get a key from Google AI Studio.
API_KEY = "YOUR_GEMINI_API_KEY"

# The file uploaded by the user
DATA_FILE = "/content/ElectricCarData_Clean.csv"

# Gemini API Endpoint
# Using gemini-2.5-flash-preview-09-2025 as the model for grounded generation
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-09-2025:generateContent?key={API_KEY}"

# Load and prepare the electric car data for context
try:
    df_cars = pd.read_csv(DATA_FILE)
    # Convert the DataFrame to a Markdown table string for easy LLM interpretation
    DATA_CONTEXT = df_cars.to_markdown(index=False, numalign="left", stralign="left")

    # Define a focused system instruction
    SYSTEM_INSTRUCTION = (
        "You are an expert Electric Vehicle (EV) Data Analyst. Your knowledge base is strictly limited "
        "to the provided CSV data context below. Your goal is to answer user questions about EV "
        "performance, range, and pricing ONLY based on this data. "
        "If the data does not contain the answer, state that clearly. "
        "The data is provided in Markdown table format."
        f"\n\n--- DATA CONTEXT ---\n{DATA_CONTEXT}\n--- END DATA CONTEXT ---\n"
    )
    print(f"Successfully loaded {DATA_FILE} with {len(df_cars)} records.")

except FileNotFoundError:
    DATA_CONTEXT = "Error: Data file not found. I cannot perform data analysis."
    SYSTEM_INSTRUCTION = "You are a helpful chatbot. I am currently experiencing a data loading error."
    print(f"Error: {DATA_FILE} not found. Running in generic chatbot mode.")
except Exception as e:
    DATA_CONTEXT = f"Error: Failed to process data file. {e}"
    SYSTEM_INSTRUCTION = "You are a helpful chatbot. I am currently experiencing a data processing error."
    print(f"Error processing data: {e}. Running in generic chatbot mode.")


# --- SECTION 3: Gemini API Call Function ---

def call_gemini_api(prompt, chat_history, system_instruction):
    """
    Makes a POST request to the Gemini API with exponential backoff.
    """

    # Convert Gradio history into the format expected by the Gemini API
    contents = []
    for user_msg, model_msg in chat_history:
        # User message
        contents.append({
            "role": "user",
            "parts": [{"text": user_msg}]
        })
        # Model message
        contents.append({
            "role": "model",
            "parts": [{"text": model_msg}]
        })

    # Append the current user prompt
    contents.append({
        "role": "user",
        "parts": [{"text": prompt}]
    })

    # Construct the payload
    payload = {
        "contents": contents,
        "systemInstruction": {
            "parts": [{"text": system_instruction}]
        },
        # Enable Google Search grounding only if we couldn't load the EV data
        "tools": [] if DATA_CONTEXT.startswith("Error") else []
    }

    # Exponential backoff logic
    max_retries = 5
    retry_delay = 1.0

    for attempt in range(max_retries):
        try:
            headers = {'Content-Type': 'application/json'}
            response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
            response.raise_for_status() # Raise exception for bad status codes (4xx or 5xx)

            result = response.json()

            # Extract generated text
            candidate = result.get('candidates', [{}])[0]
            text = candidate.get('content', {}).get('parts', [{}])[0].get('text', 'API Response Error: Could not extract text.')
            return text

        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                print(f"API request failed (Attempt {attempt+1}/{max_retries}): {e}. Retrying in {retry_delay:.2f}s...")
                time.sleep(retry_delay)
                retry_delay *= 2 # Exponential increase
            else:
                return f"API Error after multiple retries: Could not reach the service. Details: {e}"
        except json.JSONDecodeError:
            return "API Error: Failed to decode JSON response."
        except Exception as e:
            return f"An unexpected error occurred: {e}"


# --- SECTION 4: Gradio Chat Function ---

def respond(message, history):
    """
    The main chat function for Gradio.
    """
    if API_KEY == "YOUR_GEMINI_API_KEY" or not API_KEY:
        return "Please set your Gemini API Key in the `API_KEY` variable inside the Python code to enable the chatbot."

    print(f"User: {message}")

    # Call the Gemini model with the user's message, history, and the system instruction (which includes the data)
    model_response = call_gemini_api(
        prompt=message,
        chat_history=history,
        system_instruction=SYSTEM_INSTRUCTION
    )

    print(f"Gemini: {model_response}")
    return model_response

# --- SECTION 5: Gradio Interface Setup and Launch ---

# Create the Gradio ChatInterface
iface = gr.ChatInterface(
    fn=respond,
    title="Electric Car Chatbot",
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Ask about range, price, or performance data for the cars in the CSV...", container=False, scale=7),
    theme="soft",
    description="I am an expert analyst. I can answer questions about the uploaded EV data based *only* on the contents of the `ElectricCarData_Clean.csv` file.",
)

# Launch the app with share=True to get a public URL for the website output
if __name__ == "__main__":
    iface.launch(
        share=True,
        debug=True,
        # We need to set max_threads to 1 because Gradio's internal threading can cause issues
        # when calling external APIs rapidly in certain environments like Colab.
        max_threads=1
    )

Successfully loaded /content/ElectricCarData_Clean.csv with 103 records.


/tmp/ipython-input-1388069095.py:148: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500),
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://52641637544872cbf0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
